<h1 style="text-align: center">
<strong>WEB SCRAPING - REPOSITORIO DE UNSA</strong>
</h1>

**Nombre:** Oliverio Pichardo Diestra

**Descripción del caso:**
Se quiere obtener información estructurada de las tesis de pregrado y posgrado de Estadística 
Informática o de carreras similares desarrolladas a nivel  nacional, a través de los repositorios 
institucionales de las universidades.

**Especificaciones:**
1. Debido a que se menciona carreras similares y no se especifica terminología alguna para ello, 
entonces se procede a considerar lo establecido por INEI en su *Clasificador Nacional de Programas e Instituciones de 
Educación Superior Universitaria, Pedagógica, Tecnológica y Técnico Productiva, 2018*. (Ver Tabla 1)

    **Tabla 1**  
    *Programas similares a Estadística a nivel de profesional (pregrado)*
    | CÓDIGO |                      PROGRAMA                     |
    |:------:|:-------------------------------------------------:|
    | 542016 |                    Estadística                    |
    | 542026 |             Estadística e informática             |
    | 542036 |              Estadística informática              |
    | 542046 | Estadística para la gestión de servicios de salud |
    | 542056 |               Ingeniería estadística              |
    | 542066 |        Ingeniería estadística e informática       |
    | 542076 |         Ingeniería estadística informática        |
    | 542996 |           Otros programas en estadística          |

    *Nota: Adaptado de "Clasificador Nacional de Programas e Instituciones de 
    Educación Superior Universitaria, Pedagógica, Tecnológica y Técnico Productiva, 2018" por INEI.*

2. El repositorio escogido es el de la Universidad Nacional de San Agustín de Arequipa (UNSA). 
El respectivo url de dicho repositorio es:  http://repositorio.unsa.edu.pe/browse?type=dateissued **(url-principal)**


# 1. Web Scraping

## 1.1. Librerías, módulos o paquetes
A fin de realizar la extracción de datos pertinentes, se emplearán las siguientes librerías, módulos o paquetes:

In [1]:
# Cargando librerías, módulos o paquetes pertinentes
# %pip install requests
import requests
from bs4 import BeautifulSoup
import re
import itertools

## 1.2. Funciones
Se definirán dos funciones las cuales tendrán los siguientes objetivos respectivamente:
1. Extraer todas las urls de la diversas páginas de la *url-principal* en las que se encuentran las tesis de pregrado y posgrado. Tales urls serán denominadas *url-página*.
2. De cada *url-página*, extraer las urls que contienen la información específica de cada una las tesis. A estas les denominaremos *url-publicación*.

A fin de comprender mejor lo mencionado, veamos la estructura jerárquica:
1. Nivel 1 (url-principal): http://repositorio.unsa.edu.pe/browse?type=dateissued
2. Nivel 2 (url-página): Es una subpágina de la url-principal que contiene en promedio 20 publicaciones. Hasta la fecha, en total, existen 682 subpáginas.
3. Nivel 3 (url-publicación): Es una subpágina de una url-página que contiene la información específica de una determinada publicación. Hasta el momento, contiene en promedio 28 descriptores de la respectiva publicación.

### 1.2.3. Extracción de todas las url-página de la url-principal
A fin de realizar la extracción de todas las url-página de la url-principal, se identificó un patrón de url que sirve para generar todas las url-página. A esa se le denominará *url-base*.
A continuación, se define una función en la que:
1. se identifica la url-base,
2. se crea una lista de los caracteres que se deben reemplazar en el url-base para generar todas la url-página y
3. se generan las url-página mediante la sustitución de unos caracteres específicos de la url-base por los elementos la lista generada en el punto 2.

In [ ]:
# Creando una función que permite extraer todas las url-página
def all_pages(url):
    # Descargar el contenido de la url-principal
    page = requests.get(url)
    # Crear un objeto BeautifulSoup a partir del contenido de la página
    soup = BeautifulSoup(page.content, "html.parser")
    # Encontrar el tag a que contiene la url-base de las url-página
    pages = soup.find_all("a", {"class":"next-page-link"})
    # Extraer el url-base de las url-página
    pages_soup = BeautifulSoup(str(pages), 'html.parser')
    a_tags = pages_soup.find_all('a') # find all anchor elements
    base_href = ["http://repositorio.unsa.edu.pe/" + a_tag['href'] for a_tag in a_tags][0]
    # Crear lista de límites inferiores de cada url-página
    string_pages = soup.find_all("p", {"class":"pagination-info"})
    matches = re.findall("\d+", str(string_pages[0]))
    lower_limit_max = (max([int(item) for item in matches]) // 20) * 20
    lower_limits = list(range(0, lower_limit_max + 20, 20))
    # Crear lista de las url-página de la url-principal
    hrefs_all = [re.sub(r"offset=\d+", "offset=" + str(pag), base_href) for pag in lower_limits]
    return hrefs_all